In [1]:
import pandas as pd
import dataframe_image as dfi
import requests
import os
import json
import requests
import warnings
from bs4 import BeautifulSoup
os.makedirs("./Result",exist_ok=True)



# 國內成分證券ETF
https://www.twse.com.tw/zh/products/securities/etf/products/domestic.html

In [2]:


url='https://www.twse.com.tw/rwd/zh/ETF/domestic?response=json&_=1681734086399'

headers = {
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36"
}
req = requests.get(url, headers = headers,verify=False)
ETFdata = req.content.decode('utf8')
ETFdf=pd.DataFrame(json.loads(ETFdata)['data'])
ETFdf=ETFdf.dropna()
ETFdf.columns = ['證券代號', '證券簡稱','']

C:\Users\user\anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.twse.com.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(



# ETF   持股 Data

In [3]:
# https://www.moneydj.com/ETF/X/Basic/Basic0007A.xdjhtm?etfid=0050.TW
# https://www.moneydj.com/etf/eb/et305001list.djhtm?order=4&c=47
def get_erfBaseData(etfid,rtfdf):
    etfurl='https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid='+etfid+'.TW'
    etfresp = requests.get(etfurl)
    etfresp.encoding = 'utf-8'    #轉換編碼至UTF-8
    print("utl="+etfurl)
    if etfresp.status_code==200 :
        etfsoup = BeautifulSoup(etfresp.text, 'lxml')
        
        #印出表格中的所有文字內容
        for tr in etfsoup.find('table', id="Repeater1").find_all('tr'):
            td=tr.find_all('td')
            row = [i.text for i in td]
            row.append(etfid)
            if len(row)>0:
                rtfdf = rtfdf.append([row],ignore_index=True)
        for tr in etfsoup.find('table', id="Repeater2").find_all('tr'):
            td=tr.find_all('td')
            row = [i.text for i in td]
            if len(row)>0:
                row.append(etfid)
                rtfdf = rtfdf.append([row],ignore_index=True)

        rtfdf=rtfdf.dropna()
       
        return rtfdf

In [4]:
rtfdf = pd.DataFrame()
for  etfid in ETFdf['證券代號'].tolist():
    rtfdf=get_erfBaseData(etfid,rtfdf)
# 【Output】   
rtfdf.columns = ['股票名稱', '持股(千股)','比例','增減','ETF_Id']
rtfdf.to_csv(r'BaseInfo/ETFBaseData.csv', index = False, encoding='utf-8', errors='ignore')
rtfdf.to_csv(r'Result/ETFBaseDataRead.csv', index = False,encoding='big5', errors='ignore')

utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=0050.TW


C:\Users\user\AppData\Local\Temp\ipykernel_30964\1022840050.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rtfdf = rtfdf.append([row],ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_30964\1022840050.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rtfdf = rtfdf.append([row],ignore_index=True)


utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=0051.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=0052.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=0053.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=0055.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=0056.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=0057.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=006203.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=006204.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=006208.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=00690.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=00692.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=00701.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=00713.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdj

In [5]:



StockBaseData=pd.read_csv('BaseInfo/StockBaseData.csv')
ETFBaseData=pd.read_csv('BaseInfo/ETFBaseData.csv')

In [6]:
# 兩表Join
df=pd.merge(ETFBaseData, StockBaseData, how='left', left_on='股票名稱', right_on='公司簡稱')
# 處理格式問題
df['ETF_Id']='00'+df['ETF_Id'].astype(str)

#處理上市公司產業類別對應不到的狀況
df['上市公司產業類別']=df['上市公司產業類別'].fillna('未分類')
print(df.columns)

Index(['股票名稱', '持股(千股)', '比例', '增減', 'ETF_Id', '公司代號', '公司名稱', '公司簡稱', '產業別',
       '實收資本額', 'Type', '上市公司產業類別', '上櫃公司產業類別', '新編碼'],
      dtype='object')


In [7]:
# 取得 etf 對應持股佔比
df['data']=df.agg(lambda x: f"{x['股票名稱']} ( {x['比例']}%)", axis=1)
df.sort_values('比例', ascending=False)
t=df[['ETF_Id','data']]
t[t['ETF_Id']=='00713']

,ETF_Id,data
874,00713,台灣大 ( 9.61%)
875,00713,統一 ( 9.28%)
876,00713,遠傳 ( 8.76%)
877,00713,統一超 ( 5.96%)
878,00713,台新金 ( 3.58%)
879,00713,華南金 ( 3.52%)
880,00713,聯發科 ( 3.02%)
881,00713,中信金 ( 2.95%)
882,00713,玉山金 ( 2.4%)
883,00713,洋基工程 ( 2.33%)


In [8]:
keydata=df.groupby(['ETF_Id','上市公司產業類別'])['data'].apply(','.join).reset_index()
keydata

,ETF_Id,上市公司產業類別,data
0,0050,光電業,大立光 ( 0.58%)
1,0050,其他,中租-KY ( 0.29%)
2,0050,其他電子業,鴻海 ( 4.69%)
3,0050,半導體業,"台積電 ( 56.96%),聯發科 ( 5.17%),日月光投控 ( 1.25%),聯電 (..."
4,0050,塑膠工業,"南亞 ( 0.39%),台塑 ( 0.32%),台化 ( 0.22%)"
...,...,...,...
694,00962,資訊服務業,"零壹 ( 1.06%),精誠 ( 1.06%),緯軟 ( 0.19%)"
695,00962,通信網路業,"遠傳 ( 3.04%),美律 ( 1.53%),中磊 ( 1.48%),是方 ( 0.85%..."
696,00962,電子零組件業,"建準 ( 1.34%),群電 ( 0.95%),佳邦 ( 0.46%)"
697,00962,電機機械,"亞德客-KY ( 3.64%),東元 ( 2.14%)"


In [9]:
# 取得產業比例排名 by ETF_Id 上市公司產業類別

# 挑選要用的欄位
colList=['ETF_Id','股票名稱', '上市公司產業類別','持股(千股)','比例']
df_=df[colList]
# groupby
g=df_.groupby(['ETF_Id','上市公司產業類別']).sum().reset_index()
# 排序 取前三
#g.sort_values('比例', ascending=False).groupby(['ETF_Id']).head(3).sort_values(['ETF_Id', '比例'], ascending=[True, False])
# 排序(這個對未來比較有彈性)
g['row_numbers'] =g.groupby('ETF_Id')['比例'].rank(ascending=False,method='dense')
g[g['ETF_Id']=='00713']

C:\Users\user\AppData\Local\Temp\ipykernel_30964\2400789445.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  g=df_.groupby(['ETF_Id','上市公司產業類別']).sum().reset_index()


,ETF_Id,上市公司產業類別,比例,row_numbers
200,00713,光電業,3.22,10.0
201,00713,其他,3.97,8.0
202,00713,其他電子業,3.99,7.0
203,00713,化學工業,0.99,17.0
204,00713,半導體業,6.74,4.0
205,00713,建材營造,0.84,18.0
206,00713,汽車工業,1.36,15.0
207,00713,紡織纖維,1.19,16.0
208,00713,航運業,2.25,11.0
209,00713,貿易百貨,5.96,5.0


In [10]:
#轉成橫的方便看
df1=g[g['row_numbers']==1].merge(keydata,on=['ETF_Id','上市公司產業類別'])
df2=g[g['row_numbers']==2].merge(keydata,on=['ETF_Id','上市公司產業類別'])
df3=g[g['row_numbers']==3].merge(keydata,on=['ETF_Id','上市公司產業類別'])
df_result=df1.merge(df2,on='ETF_Id',suffixes=["","_2"]).merge(df3,on='ETF_Id',suffixes=["","_3"])
df_result = df_result.drop(df_result.filter(regex='row_numbers').columns, axis=1)
df_result

,ETF_Id,上市公司產業類別,比例,data,上市公司產業類別_2,比例_2,data_2,上市公司產業類別_3,比例_3,data_3
0,0050,半導體業,66.42,"台積電 ( 56.96%),聯發科 ( 5.17%),日月光投控 ( 1.25%),聯電 (...",金融保險,11.57,"富邦金 ( 1.7%),中信金 ( 1.52%),國泰金 ( 1.43%),兆豐金 ( 1....",電腦及週邊設備業,5.63,"廣達 ( 1.56%),華碩 ( 1.15%),緯創 ( 0.69%),光寶科 ( 0.48..."
1,0051,電子零組件業,13.92,"台光電 ( 2.5%),健策 ( 1.8%),川湖 ( 1.61%),嘉澤 ( 1.58%)...",半導體業,11.52,"矽力*-KY ( 1.94%),京元電子 ( 1.55%),創意 ( 1.55%),祥碩 (...",電腦及週邊設備業,10.53,"仁寶 ( 2.05%),技嘉 ( 1.74%),微星 ( 1.64%),英業達 ( 1.55..."
2,0052,半導體業,78.93,"台積電 ( 65.73%),聯發科 ( 5.9%),日月光投控 ( 1.45%),聯電 ( ...",電腦及週邊設備業,8.48,"廣達 ( 1.75%),華碩 ( 1.26%),緯創 ( 0.8%),光寶科 ( 0.56%...",其他電子業,6.27,"鴻海 ( 5.46%),可成 ( 0.33%),貿聯-KY ( 0.26%),鴻準 ( 0...."
3,0053,半導體業,65.40,"台積電 ( 52.08%),聯發科 ( 5.39%),日月光投控 ( 1.48%),聯電 (...",電腦及週邊設備業,9.10,"廣達 ( 1.87%),華碩 ( 0.97%),緯穎 ( 0.75%),緯創 ( 0.64%...",電子零組件業,8.13,"台達電 ( 2.74%),川湖 ( 0.47%),嘉澤 ( 0.42%),台光電 ( 0.4..."
4,0056,電腦及週邊設備業,23.58,"華碩 ( 3.39%),和碩 ( 2.49%),仁寶 ( 2.42%),光寶科 ( 2.35...",半導體業,18.19,"聯詠 ( 3.58%),瑞昱 ( 3.22%),聯發科 ( 2.99%),聯電 ( 2.95...",金融保險,15.39,"中信金 ( 3.27%),華南金 ( 2.74%),元大金 ( 2.49%),兆豐金 ( 2..."
5,0057,半導體業,61.63,"台積電 ( 51.97%),聯發科 ( 4.59%),日月光投控 ( 1.13%),聯電 (...",金融保險,10.96,"富邦金 ( 1.51%),中信金 ( 1.35%),國泰金 ( 1.3%),兆豐金 ( 0....",電腦及週邊設備業,6.26,"廣達 ( 1.35%),華碩 ( 0.96%),緯創 ( 0.61%),光寶科 ( 0.45..."
6,006203,半導體業,61.90,"台積電 ( 51.97%),聯發科 ( 4.7%),日月光投控 ( 1.14%),聯電 ( ...",金融保險,11.02,"富邦金 ( 1.53%),中信金 ( 1.36%),國泰金 ( 1.31%),兆豐金 ( 0...",電腦及週邊設備業,6.36,"廣達 ( 1.36%),華碩 ( 0.99%),緯創 ( 0.61%),緯穎 ( 0.46%..."
7,006204,半導體業,40.68,"台積電 ( 35.26%),聯發科 ( 4.14%),日月光投控 ( 1.28%)",金融保險,5.20,"富邦金 ( 2.12%),國泰金 ( 1.72%),中信金 ( 1.36%)",其他電子業,3.71,鴻海 ( 3.71%)
8,006208,半導體業,66.14,"台積電 ( 56.78%),聯發科 ( 5.08%),日月光投控 ( 1.25%),聯電 (...",金融保險,11.53,"富邦金 ( 1.7%),中信金 ( 1.51%),國泰金 ( 1.43%),兆豐金 ( 1....",電腦及週邊設備業,5.47,"廣達 ( 1.51%),華碩 ( 1.09%),緯創 ( 0.69%),光寶科 ( 0.48..."
9,00690,半導體業,60.39,"台積電 ( 32.2%),聯發科 ( 16.06%),聯電 ( 3.59%),聯詠 ( 2....",電子零組件業,12.32,"台達電 ( 5.96%),國巨 ( 1.84%),台光電 ( 1.39%),嘉澤 ( 1.0...",金融保險,7.81,"富邦金 ( 6.41%),上海商銀 ( 1.4%)"


In [19]:

# export png
df_styled = df_result.style.background_gradient()
df_styled = df_result.style.set_properties(subset=['data','data_2','data_3'], **{'width': '600px'})
dfi.export(df_styled, "Result/mytable.png")

In [22]:
df_styled = (
    df_result.style
    .background_gradient()
    .set_properties(subset=['data', 'data_2', 'data_3'], **{'width': '600px'})
)

dfi.export(df_styled, "Result/mytable.png")